In [146]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

In [147]:
rating = pd.read_csv('/home/rating_baru.csv')
place = pd.read_csv('/home/place_malang.csv')

In [148]:
rating.head(2)

,User_Id,Place_Id,Place_Ratings
0,1,438,2
1,20,438,2


['.config',
 'recommender_model.tflite',
 'recommender_model',
 'recommender_model.zip',
 'sample_data']

Isi folder yang telah diekstrak:
['.config', 'recommender_model.tflite', 'recommender_model', 'recommender_model.zip', 'sample_data']


In [56]:
#model = tf.keras.models.load_model("/content/recommender_model")


In [150]:
model = tf.lite.Interpreter(model_path="/content/recommender_model.tflite")
model.allocate_tensors()

In [151]:
df = rating.copy()
df.head()

,User_Id,Place_Id,Place_Ratings
0,1,438,2
1,20,438,2
2,39,438,3
3,58,438,2
4,77,438,5


MELAKUKAN ENCODING


1. MEMBUAT FUNGSI ENCODING

In [152]:
def dict_encoder(col, data=df):

  # Mengubah kolom suatu dataframe menjadi list tanpa nilai yang sama
  unique_val = data[col].unique().tolist()

  # Melakukan encoding value kolom suatu dataframe ke angka
  val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}

  # Melakukan proses encoding angka ke value dari kolom suatu dataframe
  val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
  return val_to_val_encoded, val_encoded_to_val

2. Encoding dan Mapping Kolom User

In [153]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('User_Id')

# Mapping User_Id ke dataframe
df['user'] = df['User_Id'].map(user_to_user_encoded)

3. Encoding dan Mapping Kolom Place

In [154]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Place_Id')

# Mapping Place_Id ke dataframe place
df['place'] = df['Place_Id'].map(place_to_place_encoded)

In [155]:
# Mendapatkan jumlah user dan place
num_users, num_place = len(user_to_user_encoded), len(place_to_place_encoded)

# Mengubah rating menjadi nilai float
df['Place_Ratings'] = df['Place_Ratings'].values.astype(np.float32)

# Mendapatkan nilai minimum dan maksimum rating
min_rating, max_rating = min(df['Place_Ratings']), max(df['Place_Ratings'])

print(f'Number of User: {num_users}, Number of Place: {num_place}, Min Rating: {min_rating}, Max Rating: {max_rating}')

Number of User: 300, Number of Place: 57, Min Rating: 1.0, Max Rating: 5.0


In [156]:
# Mengacak dataset
df = df.sample(frac=1, random_state=42)
df.head(2)

,User_Id,Place_Id,Place_Ratings,user,place
203,258,476,2.0,203,38
266,232,488,3.0,266,50


 Menyiapkan DataFrame untuk Menampilkan Hasil Rekomendasi

In [157]:
place_df = place[['Place_Id','Place_Name','Category','Rating','Price']]
place_df.columns = ['id','place_name','category','rating','price']
df = rating.copy()

Menyiapkan contoh User untuk Menampilkan Rekomendasi

MENCARI USER

In [172]:
# Mengambil sample user
user_id = df.User_Id.sample(1).iloc[0]
place_visited_by_user = df[df.User_Id == user_id]

Mencari Lokasi yang belum Dikunjungi User

In [173]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Place_Id.values)]['id']
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)

place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [174]:
print(user_id)
print(place_not_visited)

188
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56]]


In [160]:
input_details = model.get_input_details()
output_details = model.get_output_details()

In [161]:
model.allocate_tensors()

In [162]:
# Assuming user_place_array currently has multiple rows
if len(user_place_array.shape) > 1:
    # Extract the first row (representing a single user-place combination)
    user_place_array = user_place_array[0]

# Reshape to ensure it's a 1D array with 2 elements
user_place_array = user_place_array.reshape(1, -1)

model.set_tensor(input_details[0]['index'], user_place_array)

Menampilkan Hasil Rekomendasi untuk User

In [163]:

# Run inference using TFLite model
model.invoke()  # Assuming you've already called allocate_tensors()

# Get output tensor data (assuming ratings are the first output)
output_data = model.get_tensor(output_details[0]['index'])
ratings = output_data.flatten()  # Assuming output is a 1D array representing ratings

# Extract top 7 recommendations
top_ratings_indices = ratings.argsort()[-7:][::-1]
recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]



print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)

top_place_user = (
    place_visited_by_user.sort_values(
        by = 'Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)

place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)

recommended_place = place_df[place_df['id'].isin(recommended_place_ids)]
for row, i in zip(recommended_place.itertuples(), range(1,8)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Harga Tiket Masuk ', row.price, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)

Daftar rekomendasi untuk: User 187

------------------------------------------------------------
Tempat dengan rating wisata paling tinggi dari user
------------------------------------------------------------
Pantai Bujulmati : Pantai

------------------------------------------------------------
Top 7 place recommendation
------------------------------------------------------------
1 . Jatim Park 1 
     Taman Rekreasi , Harga Tiket Masuk  nan , Rating Wisata  4.5 

